In [9]:
import absl
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

logger = absl.logging.get_absl_logger()
logger.setLevel('ERROR')

In [10]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()

# Loading data

In [11]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_examples, test_examples = dataset['train'], dataset['test']

Encode string

In [12]:
sample_string = 'Hello World.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))


Encoded string is [4025, 222, 2307, 1829, 7975]
The original string: "Hello World."


In [16]:
for index in encoded_string:
    print('{} ----> {}'.format(index, encoder.decode([index])))


4025 ----> Hell
222 ----> o 
2307 ----> Wor
1829 ----> ld
7975 ----> .


Preparing data

In [14]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [21]:
train_dataset = train_examples.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
test_dataset = train_examples.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)

# Create model

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [25]:
history = model.fit(train_dataset, epochs=3,
                    validation_data=test_dataset, 
                    validation_steps=30)

Epoch 1/3
391/391 [==============================] - 449s 1s/step - loss: 0.2143 - accuracy: 0.9203 - val_loss: 0.1672 - val_accuracy: 0.9380
Epoch 2/3
391/391 [==============================] - 468s 1s/step - loss: 0.1885 - accuracy: 0.9314 - val_loss: 0.1622 - val_accuracy: 0.9432
Epoch 3/3
391/391 [==============================] - 450s 1s/step - loss: 0.1642 - accuracy: 0.9429 - val_loss: 0.1501 - val_accuracy: 0.9359


In [26]:
test_loss, test_accuracy = model.evaluate(test_dataset)

391/391 [==============================] - 102s 260ms/step - loss: 0.1459 - accuracy: 0.9395


In [28]:
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_accuracy))

Test Loss: 0.14594854414463043
Test Accuracy: 0.9394800066947937
